## Maps Neighborhoods in Toronto

### import Libraries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


### Load DataFrame using Pandas with CSV

In [2]:
df_data = pd.read_csv("DatasetPostCodeToronto.csv")
df_data.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Delete Borough with value equal "not assigned"

In [4]:
Notassigned = df_data[ df_data['Borough'] == "Not assigned" ].index
 
# Delete these row indexes from dataFrame
df_data.drop(Notassigned , inplace=True)

In [5]:
df_data.head()

,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
df_data.PostCode.unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

### Create GroupBy with postcode and union Neighbourhood

In [7]:
df_postcode=df_data.groupby("PostCode")

In [8]:
df2=df_postcode['Neighbourhood'].apply(','.join).to_frame().reset_index()
df2.head()

,PostCode,Neighbourhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [9]:
df1=df_postcode.Borough.unique().to_frame().reset_index()
df1.head()

,PostCode,Borough
0,M1B,[Scarborough]
1,M1C,[Scarborough]
2,M1E,[Scarborough]
3,M1G,[Scarborough]
4,M1H,[Scarborough]


In [10]:
df_clean=pd.merge(df1, df2, on='PostCode')
df_clean.head()

,PostCode,Borough,Neighbourhood
0,M1B,[Scarborough],"Rouge,Malvern"
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill"
3,M1G,[Scarborough],Woburn
4,M1H,[Scarborough],Cedarbrae


In [11]:
df_clean.shape

(103, 3)

In [12]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### concatenate Dataframe Postcode with DataFrame Coordenate

In [13]:
df_final=pd.concat([df_clean,df_coord], axis=1, join="inner")

In [14]:
df_final.head()

,PostCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,[Scarborough],"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,[Scarborough],Woburn,M1G,43.770992,-79.216917
4,M1H,[Scarborough],Cedarbrae,M1H,43.773136,-79.239476


In [15]:
#Delete column duplicate Postcode
df_final.drop("Postal Code", axis=1)

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,[Scarborough],"Rouge,Malvern",43.806686,-79.194353
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,[Scarborough],Woburn,43.770992,-79.216917
4,M1H,[Scarborough],Cedarbrae,43.773136,-79.239476
5,M1J,[Scarborough],Scarborough Village,43.744734,-79.239476
6,M1K,[Scarborough],"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,[Scarborough],"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,[Scarborough],"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,[Scarborough],"Birch Cliff,Cliffside West",43.692657,-79.264848


### Select only "East Toronto"

In [39]:
toronto_data = df_final[df_final['Borough'] == 'East Toronto'].reset_index(drop=True)
toronto_data.head()

,PostCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,[East Toronto],The Beaches,M4E,43.676357,-79.293031
1,M4K,[East Toronto],"The Danforth West,Riverdale",M4K,43.679557,-79.352188
2,M4L,[East Toronto],"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
3,M4M,[East Toronto],Studio District,M4M,43.659526,-79.340923
4,M7Y,[East Toronto],Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558


### I get coordante about Toronto and Print

In [33]:
address = 'East Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create Map Toronto

In [38]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto